In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/hugingface/fake_dataset.zip'

Archive:  /content/drive/MyDrive/hugingface/fake_dataset.zip
  inflating: fake_dataset.csv        


In [57]:
import torch
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score

In [39]:
train = pd.read_csv('/content/drive/MyDrive/hugingface/train.csv', index_col=0)
train

,description_x,description_y,ticker_x,ticker_y,same_security
0,first trust dow jones internet,first trust dj internet idx,FDN,FDN,True
1,schwab intl large company index etf,schwab strategic tr fundamental intl large co ...,FNDF,FNDF,True
2,vanguard small cap index adm,vanguard small-cap index fund inst,VSMAX,VSCIX,False
3,duke energy corp new com new isin #us4 sedol #...,duke energy corp new com new isin #us26441c204...,DUK,DUK,True
4,visa inc class a,visa inc.,V,V,True
...,...,...,...,...,...
2137,delta air lines inc com,delta air lines inc dela new,DAL,DAL,True
2138,cognizant tech solutns a,cognizant tech solutions corp,CTSH,CTSH,True
2139,pimco investment grade corporate bond fund - c...,fid conservative income bond fund,PBDAX,FCONX,False
2140,eli lilly & co com,lilly eli &co com npv,LLY,LLY,True


In [16]:
test = pd.read_csv('/content/drive/MyDrive/hugingface/test.csv', index_col=0)

In [17]:
test

,description_x,description_y,same_security
test_id,,,
0,semtech corp,semtech corporation,NaN
1,vanguard mid cap index,vanguard midcap index - a,NaN
2,spdr gold trust gold shares,spdr gold trust spdr gold shares,NaN
3,vanguard total bond index adm,vanguard total bond market index,NaN
4,oakmark international fund class i,oakmark international cl i,NaN
...,...,...,...
511,at&t inc com isin #us00206r1023 sedol #2831811,franklin res inc com isin #us8 sedol #2350684,NaN
512,vanguard total international bond index etf,vanguard total international bond et,NaN
513,wisdomtree japan hedged equity -,wisdomtree japan hedged eq,NaN


In [22]:
# Load model directly
from transformers import AutoTokenizer, AutoModel

# Загрузка модели и токенизатора
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
model = AutoModel.from_pretrained('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2').to(device)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [23]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(250037, 384, padding_idx=0)
    (position_embeddings): Embedding(512, 384)
    (token_type_embeddings): Embedding(2, 384)
    (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=384, out_features=384, bias=True)
            (key): Linear(in_features=384, out_features=384, bias=True)
            (value): Linear(in_features=384, out_features=384, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=384, out_features=384, bias=True)
            (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=Fals

In [24]:
# Функция для получения sentence embedding
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

In [41]:
# Подготовка списка текстов для сравнения
texts1_test = test['description_x'].fillna('').tolist()
texts2_test = test['description_y'].fillna('').tolist()

In [42]:
# Подготовка списка текстов для сравнения
texts1 = train['description_x'].fillna('').tolist()
texts2 = train['description_y'].fillna('').tolist()
y_true = train['same_security'].astype(int).values  # True=1, False=0

In [43]:
texts1

['first trust dow jones internet',
 'schwab intl large company index etf',
 'vanguard small cap index adm',
 'duke energy corp new com new isin #us4 sedol #b7jzsk0',
 'visa inc class a',
 'ford motor co new div: 0.600',
 'united states steel corp',
 'vanguard total international bond index etf',
 'schwab strategic tr us sml c',
 'mfs value fd cl i',
 'oppenheimer dev markets class a',
 'genomic health',
 'spdr barclays international etf',
 'kraft heinz company',
 'vanguard total bond market index fund admiral shares',
 'vanguard intermediate-term investment-grade fund investor shares',
 'baidu inc spon adr',
 'novartis ag- reg',
 'seaspan corp',
 'vanguard total bond market index admiral',
 'mfs muni high income fd cl i',
 'metlife inc com',
 'russell gbl real estate securities cl s',
 'vanguard short-term investment-grade fund admiral shares',
 'spdr s&p 500 etf trust',
 'pimco total return a',
 'intel corp',
 'vanguard 500 index fund - admiral class',
 'paypal holdings inc',
 'pim to

In [44]:
texts2

['first trust dj internet idx',
 'schwab strategic tr fundamental intl large co index etf',
 'vanguard small-cap index fund inst',
 'duke energy corp new com new isin #us26441c2044 sedol #b7jzs',
 'visa inc.',
 'ford motor co',
 'united sts stl cp (new)',
 'vanguard total intl bond index etf',
 'schwab us small cap etf',
 'mfs value fund cl i',
 'oppenheimer developing mkts fd cl a',
 'genomic health inc com',
 'spdr barclays capital international trea',
 'kraft heinz co com',
 'vanguard total bond market idx instl pls',
 'vanguard interm-term invmt-grade inv',
 'baidu inc spon adr rep a',
 'novartis ag',
 'seaspan corporation cmn',
 'vanguard bond index total mkt investor',
 'mfs municipal high cl i',
 'metlife incorporated',
 'vy clarion real estate port srv',
 'vanguard short-term invest-grade',
 'spdr s&p 500 etf iv',
 'fund: pimco total return admin',
 'intel corporation',
 'vanguard 500 idx adm',
 'paypal holdings inco',
 'pimco total return fund - class r',
 'vanguard small cap 

In [45]:
# Получение sentence embeddings и расчет косинусного сходства
embeddings1, embeddings2 = [], []
for text1, text2 in tqdm(zip(texts1, texts2), total=len(texts1)):
    encoded_input1 = tokenizer(text1, padding=True, truncation=True, return_tensors='pt').to(device)
    encoded_input2 = tokenizer(text2, padding=True, truncation=True, return_tensors='pt').to(device)

    with torch.no_grad():
        model_output1 = model(**encoded_input1)
        model_output2 = model(**encoded_input2)

    emb1 = mean_pooling(model_output1, encoded_input1['attention_mask']).cpu()
    emb2 = mean_pooling(model_output2, encoded_input2['attention_mask']).cpu()

    embeddings1.append(emb1)
    embeddings2.append(emb2)

100%|██████████| 2142/2142 [02:45<00:00, 12.98it/s]


In [46]:
# Конвертация в матрицу и расчет косинусного сходства
embeddings1 = torch.stack(embeddings1).squeeze()
embeddings2 = torch.stack(embeddings2).squeeze()

In [47]:
embeddings1

tensor([[-0.2982, -0.1107, -0.4760,  ..., -0.2584,  0.0850, -0.0017],
        [-0.1297, -0.1486, -0.2017,  ..., -0.3543,  0.0839,  0.0723],
        [-0.2503,  0.2520,  0.0514,  ..., -0.0955, -0.2464,  0.2735],
        ...,
        [-0.2630, -0.0883, -0.3176,  ..., -0.3717, -0.1043,  0.1362],
        [-0.2267, -0.3795,  0.0082,  ..., -0.2480,  0.2150,  0.0321],
        [-0.1359,  0.2387, -0.1706,  ..., -0.2049, -0.0725,  0.0573]])

In [48]:
embeddings1.shape

torch.Size([2142, 384])

In [49]:
embeddings2

tensor([[-0.2158, -0.0514, -0.3378,  ..., -0.2038,  0.2119, -0.2488],
        [-0.2194,  0.1033, -0.1255,  ..., -0.1493, -0.2458,  0.0872],
        [-0.0832,  0.0570, -0.0376,  ..., -0.2624,  0.0215,  0.0462],
        ...,
        [-0.2264,  0.0128, -0.7429,  ..., -0.5625,  0.3572,  0.1079],
        [-0.2541, -0.3104,  0.0024,  ..., -0.1523,  0.0664, -0.2637],
        [-0.0441, -0.0984, -0.0167,  ..., -0.1535,  0.0518,  0.0231]])

In [50]:
embeddings2.shape

torch.Size([2142, 384])

In [51]:
similarity_scores = cosine_similarity(embeddings1, embeddings2)

In [54]:
similarity_scores

array([[0.7064729 , 0.19706863, 0.31719327, ..., 0.3403293 , 0.3470533 ,
        0.25570965],
       [0.24347794, 0.81460416, 0.56637913, ..., 0.34498516, 0.30975923,
        0.31109464],
       [0.14106971, 0.4143805 , 0.7216606 , ..., 0.2392192 , 0.22384486,
        0.21607387],
       ...,
       [0.15205139, 0.4461647 , 0.50057733, ..., 0.6964675 , 0.25810194,
        0.23478964],
       [0.2534691 , 0.19004148, 0.22591008, ..., 0.26016918, 0.881493  ,
        0.04371923],
       [0.22719464, 0.489818  , 0.2843748 , ..., 0.27544552, 0.29098982,
        0.5849346 ]], dtype=float32)

In [55]:
similarity_scores.diagonal()

array([0.7064729 , 0.81460416, 0.7216606 , ..., 0.6964675 , 0.881493  ,
       0.5849346 ], dtype=float32)

In [58]:
# Определение предсказания по порогу
threshold = 0.7  # Подберите порог для вашей задачи
y_pred = (similarity_scores.diagonal() > threshold).astype(int)

# Рассчет accuracy
accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.646125116713352


In [59]:
y_pred

array([1, 1, 1, ..., 0, 1, 0])

In [62]:
train1 = train.copy()
train1

,description_x,description_y,ticker_x,ticker_y,same_security
0,first trust dow jones internet,first trust dj internet idx,FDN,FDN,True
1,schwab intl large company index etf,schwab strategic tr fundamental intl large co ...,FNDF,FNDF,True
2,vanguard small cap index adm,vanguard small-cap index fund inst,VSMAX,VSCIX,False
3,duke energy corp new com new isin #us4 sedol #...,duke energy corp new com new isin #us26441c204...,DUK,DUK,True
4,visa inc class a,visa inc.,V,V,True
...,...,...,...,...,...
2137,delta air lines inc com,delta air lines inc dela new,DAL,DAL,True
2138,cognizant tech solutns a,cognizant tech solutions corp,CTSH,CTSH,True
2139,pimco investment grade corporate bond fund - c...,fid conservative income bond fund,PBDAX,FCONX,False
2140,eli lilly & co com,lilly eli &co com npv,LLY,LLY,True


In [65]:
train1['same_security'] = y_pred
train1

,description_x,description_y,ticker_x,ticker_y,same_security
0,first trust dow jones internet,first trust dj internet idx,FDN,FDN,1
1,schwab intl large company index etf,schwab strategic tr fundamental intl large co ...,FNDF,FNDF,1
2,vanguard small cap index adm,vanguard small-cap index fund inst,VSMAX,VSCIX,1
3,duke energy corp new com new isin #us4 sedol #...,duke energy corp new com new isin #us26441c204...,DUK,DUK,1
4,visa inc class a,visa inc.,V,V,0
...,...,...,...,...,...
2137,delta air lines inc com,delta air lines inc dela new,DAL,DAL,1
2138,cognizant tech solutns a,cognizant tech solutions corp,CTSH,CTSH,0
2139,pimco investment grade corporate bond fund - c...,fid conservative income bond fund,PBDAX,FCONX,0
2140,eli lilly & co com,lilly eli &co com npv,LLY,LLY,1
